# Synchronous Hybrid Neuroevolution Notebook

Hello Mr. Carlos!

This notebook implements the complete hybrid neuroevolution process synchronously, without the need for databases or external Kafka services. The system combines genetic algorithms with convolutional neural networks to evolve optimal architectures.

## Main Features:
- **Hybrid genetic algorithm**: Combines architecture and weight evolution
- **Synchronous processing**: Complete workflow executed in a single session
- **Configurable dataset**: Supports MNIST by default or custom dataset
- **Intelligent stopping criteria**: By target fitness or maximum generations
- **Complete visualization**: Shows progress and final best architecture

## Objectives:
1. Create initial population of CNN architectures
2. Evaluate fitness of each individual
3. Select best architectures (top 50%)
4. Apply crossover and mutation to create new generation
5. Repeat process until convergence
6. Display the best architecture found

## 1. Required Libraries Import

In [ ]:
# Install all necessary libraries
import subprocess
import sys

def install_package(package):
    """Installs a package using pip if not available."""
    try:
        __import__(package.split('==')[0].split('[')[0])
        print(f"OK {package.split('==')[0]} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"OK {package} installed correctly")

# List of required packages
required_packages = [
    "torch>=2.0.0",
    "torchvision>=0.15.0",
    "numpy>=1.21.0",
    "matplotlib>=3.5.0",
    "seaborn>=0.11.0",
    "tqdm>=4.64.0",
    "jupyter>=1.0.0",
    "ipywidgets>=8.0.0"
]

print("Starting dependency installation for Hybrid Neuroevolution...")
print("=" * 60)

for package in required_packages:
    install_package(package)

print("\nAll dependencies have been verified/installed")
print("Restart the kernel if this is the first time installing torch")
print("=" * 60)

# Verify PyTorch installation
try:
    import torch
    print(f"\nPyTorch {torch.__version__} installed correctly")
    print(f"CUDA available: {'Yes' if torch.cuda.is_available() else 'No'}")
    if torch.cuda.is_available():
        print(f"GPU detected: {torch.cuda.get_device_name(0)}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")
except ImportError:
    print("ERROR: PyTorch could not be installed correctly")
    print("Try installing manually with: pip install torch torchvision")

In [ ]:
# Main imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Scientific libraries
import numpy as np
import random
import copy
import json
import os
from typing import Dict, List, Tuple, Any
from datetime import datetime
import uuid

# Visualization and progress
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Configure logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Configure device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device configured: {device}")
print(f"PyTorch version: {torch.__version__}")

# Suppress unnecessary warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

## 2. System Configuration and Parameters

In [ ]:
# Main genetic algorithm configuration
CONFIG = {
    # Genetic algorithm parameters
    'population_size': 8,           # Population size
    'max_generations': 30,          # Maximum number of generations
    'fitness_threshold': 99.9,      # Target fitness (% accuracy)
    'mutation_rate': 0.15,          # Mutation rate
    'crossover_rate': 0.8,          # Crossover rate
    'elite_percentage': 0.5,        # Elite percentage to preserve
    
    # Dataset parameters
    'num_channels': 1,              # Input channels (1=grayscale, 3=RGB)
    'px_h': 28,                     # Image height
    'px_w': 28,                     # Image width
    'num_classes': 10,              # Number of classes
    'batch_size': 64,               # Batch size
    'test_split': 0.2,              # Validation percentage
    
    # Training parameters
    'num_epochs': 2,                # Training epochs per evaluation
    'learning_rate': 0.001,         # Base learning rate
    'early_stopping_patience': 50,  # Maximum batches for quick evaluation
    
    # Allowed architecture range
    'min_conv_layers': 1,
    'max_conv_layers': 4,
    'min_fc_layers': 1,
    'max_fc_layers': 3,
    'min_filters': 8,
    'max_filters': 128,
    'min_fc_nodes': 32,
    'max_fc_nodes': 512,
    
    # Data configuration
    'dataset_path': None,           # Custom dataset path (None = MNIST)
    'use_custom_dataset': False,    # Whether to use custom dataset
}

# Activation function mapping
ACTIVATION_FUNCTIONS = {
    'relu': nn.ReLU,
    'leaky_relu': nn.LeakyReLU,
    'tanh': nn.Tanh,
    'sigmoid': nn.Sigmoid,
    'selu': nn.SELU,
}

# Optimizer mapping
OPTIMIZERS = {
    'adam': optim.Adam,
    'adamw': optim.AdamW,
    'sgd': optim.SGD,
    'rmsprop': optim.RMSprop,
}

print("Configuration loaded:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")
print(f"\nAvailable activation functions: {list(ACTIVATION_FUNCTIONS.keys())}")
print(f"Available optimizers: {list(OPTIMIZERS.keys())}")

## 3. Dataset Loading and Preprocessing

In [ ]:
def load_dataset(config: dict) -> Tuple[DataLoader, DataLoader]:
    """
    Loads the dataset according to configuration.
    Returns train_loader and test_loader.
    """
    
    if config['use_custom_dataset'] and config['dataset_path']:
        print(f"Loading custom dataset from: {config['dataset_path']}")
        
        # Transformations for custom dataset
        transform = transforms.Compose([
            transforms.Resize((config['px_h'], config['px_w'])),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)) if config['num_channels'] == 1 else 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        # Load dataset from folders organized by class
        full_dataset = datasets.ImageFolder(root=config['dataset_path'], transform=transform)
        
        # Split into train and test
        train_size = int((1 - config['test_split']) * len(full_dataset))
        test_size = len(full_dataset) - train_size
        train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
        
        print(f"Custom dataset loaded:")
        print(f"   Classes found: {len(full_dataset.classes)}")
        print(f"   Total samples: {len(full_dataset)}")
        
    else:
        print("Loading default MNIST dataset...")
        
        # Transformations for MNIST
        transform = transforms.Compose([
            transforms.Resize((config['px_h'], config['px_w'])),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
        
        # Load MNIST
        train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
        test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
        
        print(f"MNIST dataset loaded:")
        print(f"   Classes: {len(train_dataset.classes)}")
        print(f"   Training samples: {len(train_dataset)}")
        print(f"   Test samples: {len(test_dataset)}")
    
    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config['batch_size'], 
        shuffle=True,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=config['batch_size'], 
        shuffle=False,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    return train_loader, test_loader

# Load the dataset
train_loader, test_loader = load_dataset(CONFIG)

# Get a sample to verify dimensions
sample_batch = next(iter(train_loader))
sample_data, sample_labels = sample_batch
print(f"\nDataset loaded successfully:")
print(f"   Batch shape: {sample_data.shape}")
print(f"   Data type: {sample_data.dtype}")
print(f"   Device: {sample_data.device}")
print(f"   Value range: [{sample_data.min():.3f}, {sample_data.max():.3f}]")

## 4. Neural Network Architecture Definition

In [ ]:
class EvolvableCNN(nn.Module):
    """
    Evolvable CNN class that can be dynamically configured
    according to genome parameters.
    """
    
    def __init__(self, genome: dict, config: dict):
        super(EvolvableCNN, self).__init__()
        self.genome = genome
        self.config = config
        
        # Build convolutional layers
        self.conv_layers = self._build_conv_layers()
        
        # Calculate output size after convolutions
        self.conv_output_size = self._calculate_conv_output_size()
        
        # Build fully connected layers
        self.fc_layers = self._build_fc_layers()
        
    def _build_conv_layers(self) -> nn.ModuleList:
        """Builds convolutional layers according to genome."""
        layers = nn.ModuleList()
        
        in_channels = self.config['num_channels']
        
        for i in range(self.genome['num_conv_layers']):
            out_channels = self.genome['filters'][i]
            kernel_size = self.genome['kernel_sizes'][i]
            
            # Convolutional layer
            conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
            layers.append(conv)
            
            # Batch normalization
            layers.append(nn.BatchNorm2d(out_channels))
            
            # Activation function
            activation_name = self.genome['activations'][i % len(self.genome['activations'])]
            activation_func = ACTIVATION_FUNCTIONS[activation_name]()
            layers.append(activation_func)
            
            # Max pooling (except in last layer)
            if i < self.genome['num_conv_layers'] - 1:
                layers.append(nn.MaxPool2d(2, 2))
            else:
                layers.append(nn.MaxPool2d(2, 1))  # Stride 1 in last layer
            
            in_channels = out_channels
            
        return layers
    
    def _calculate_conv_output_size(self) -> int:
        """Calculates output size after convolutional layers."""
        # Create dummy tensor to calculate size
        dummy_input = torch.zeros(1, self.config['num_channels'], 
                                 self.config['px_h'], self.config['px_w'])
        
        # Pass through convolutional layers
        x = dummy_input
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten and get size
        return x.view(-1).shape[0]
    
    def _build_fc_layers(self) -> nn.ModuleList:
        """Builds fully connected layers."""
        layers = nn.ModuleList()
        
        input_size = self.conv_output_size
        
        for i in range(self.genome['num_fc_layers']):
            output_size = self.genome['fc_nodes'][i]
            
            # Linear layer
            layers.append(nn.Linear(input_size, output_size))
            
            # Dropout if not last layer
            if i < self.genome['num_fc_layers'] - 1:
                layers.append(nn.Dropout(self.genome['dropout_rate']))
            
            input_size = output_size
        
        # Final classification layer
        layers.append(nn.Linear(input_size, self.config['num_classes']))
        
        return layers
    
    def forward(self, x):
        """Forward pass of the network."""
        # Convolutional layers
        for layer in self.conv_layers:
            x = layer(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        for i, layer in enumerate(self.fc_layers):
            x = layer(x)
            # Apply activation except on last layer
            if i < len(self.fc_layers) - 1 and not isinstance(layer, nn.Dropout):
                x = F.relu(x)
        
        return x
    
    def get_architecture_summary(self) -> str:
        """Returns an architecture summary."""
        summary = []
        summary.append(f"Conv Layers: {self.genome['num_conv_layers']}")
        summary.append(f"Filters: {self.genome['filters']}")
        summary.append(f"Kernel Sizes: {self.genome['kernel_sizes']}")
        summary.append(f"FC Layers: {self.genome['num_fc_layers']}")
        summary.append(f"FC Nodes: {self.genome['fc_nodes']}")
        summary.append(f"Activations: {self.genome['activations']}")
        summary.append(f"Dropout: {self.genome['dropout_rate']:.3f}")
        summary.append(f"Optimizer: {self.genome['optimizer']}")
        summary.append(f"Learning Rate: {self.genome['learning_rate']:.4f}")
        return " | ".join(summary)

print("EvolvableCNN class defined correctly")

## 5. Genetic Algorithm Components

In [ ]:
def create_random_genome(config: dict) -> dict:
    """Creates a random genome within specified ranges."""
    
    # Number of layers
    num_conv_layers = random.randint(config['min_conv_layers'], config['max_conv_layers'])
    num_fc_layers = random.randint(config['min_fc_layers'], config['max_fc_layers'])
    
    # Filters for each convolutional layer
    filters = []
    for _ in range(num_conv_layers):
        filters.append(random.randint(config['min_filters'], config['max_filters']))
    
    # Kernel sizes
    kernel_sizes = []
    for _ in range(num_conv_layers):
        kernel_sizes.append(random.choice([3, 5, 7]))
    
    # Nodes in fully connected layers
    fc_nodes = []
    for _ in range(num_fc_layers):
        fc_nodes.append(random.randint(config['min_fc_nodes'], config['max_fc_nodes']))
    
    # Activation functions for each layer
    activations = []
    for _ in range(max(num_conv_layers, num_fc_layers)):
        activations.append(random.choice(list(ACTIVATION_FUNCTIONS.keys())))
    
    # Other parameters
    dropout_rate = random.uniform(0.1, 0.5)
    learning_rate = random.choice([0.001, 0.0001, 0.01, 0.005])
    optimizer = random.choice(list(OPTIMIZERS.keys()))
    
    genome = {
        'num_conv_layers': num_conv_layers,
        'num_fc_layers': num_fc_layers,
        'filters': filters,
        'kernel_sizes': kernel_sizes,
        'fc_nodes': fc_nodes,
        'activations': activations,
        'dropout_rate': dropout_rate,
        'learning_rate': learning_rate,
        'optimizer': optimizer,
        'fitness': 0.0,
        'id': str(uuid.uuid4())[:8]
    }
    
    return genome

def mutate_genome(genome: dict, config: dict) -> dict:
    """Applies mutation to a genome."""
    mutated_genome = copy.deepcopy(genome)
    mutation_rate = config['mutation_rate']
    
    # Mutate number of convolutional layers
    if random.random() < mutation_rate:
        mutated_genome['num_conv_layers'] = random.randint(
            config['min_conv_layers'], config['max_conv_layers'])
        # Adjust related lists
        num_conv = mutated_genome['num_conv_layers']
        mutated_genome['filters'] = mutated_genome['filters'][:num_conv]
        mutated_genome['kernel_sizes'] = mutated_genome['kernel_sizes'][:num_conv]
        
        # Fill if necessary
        while len(mutated_genome['filters']) < num_conv:
            mutated_genome['filters'].append(
                random.randint(config['min_filters'], config['max_filters']))
        while len(mutated_genome['kernel_sizes']) < num_conv:
            mutated_genome['kernel_sizes'].append(random.choice([3, 5, 7]))
    
    # Mutate filters
    for i in range(len(mutated_genome['filters'])):
        if random.random() < mutation_rate:
            mutated_genome['filters'][i] = random.randint(
                config['min_filters'], config['max_filters'])
    
    # Mutate kernel sizes
    for i in range(len(mutated_genome['kernel_sizes'])):
        if random.random() < mutation_rate:
            mutated_genome['kernel_sizes'][i] = random.choice([3, 5, 7])
    
    # Mutate number of FC layers
    if random.random() < mutation_rate:
        mutated_genome['num_fc_layers'] = random.randint(
            config['min_fc_layers'], config['max_fc_layers'])
        # Adjust FC nodes
        num_fc = mutated_genome['num_fc_layers']
        mutated_genome['fc_nodes'] = mutated_genome['fc_nodes'][:num_fc]
        while len(mutated_genome['fc_nodes']) < num_fc:
            mutated_genome['fc_nodes'].append(
                random.randint(config['min_fc_nodes'], config['max_fc_nodes']))
    
    # Mutate FC nodes
    for i in range(len(mutated_genome['fc_nodes'])):
        if random.random() < mutation_rate:
            mutated_genome['fc_nodes'][i] = random.randint(
                config['min_fc_nodes'], config['max_fc_nodes'])
    
    # Mutate activation functions
    for i in range(len(mutated_genome['activations'])):
        if random.random() < mutation_rate:
            mutated_genome['activations'][i] = random.choice(
                list(ACTIVATION_FUNCTIONS.keys()))
    
    # Mutate dropout
    if random.random() < mutation_rate:
        mutated_genome['dropout_rate'] = random.uniform(0.1, 0.5)
    
    # Mutate learning rate
    if random.random() < mutation_rate:
        mutated_genome['learning_rate'] = random.choice([0.001, 0.0001, 0.01, 0.005])
    
    # Mutate optimizer
    if random.random() < mutation_rate:
        mutated_genome['optimizer'] = random.choice(list(OPTIMIZERS.keys()))
    
    # New ID for mutated genome
    mutated_genome['id'] = str(uuid.uuid4())[:8]
    mutated_genome['fitness'] = 0.0  # Reset fitness
    
    return mutated_genome

def crossover_genomes(parent1: dict, parent2: dict, config: dict) -> Tuple[dict, dict]:
    """Performs crossover between two genomes."""
    if random.random() > config['crossover_rate']:
        return copy.deepcopy(parent1), copy.deepcopy(parent2)
    
    child1 = copy.deepcopy(parent1)
    child2 = copy.deepcopy(parent2)
    
    # Crossover scalar parameters
    if random.random() < 0.5:
        child1['num_conv_layers'], child2['num_conv_layers'] = \
            child2['num_conv_layers'], child1['num_conv_layers']
    
    if random.random() < 0.5:
        child1['num_fc_layers'], child2['num_fc_layers'] = \
            child2['num_fc_layers'], child1['num_fc_layers']
    
    if random.random() < 0.5:
        child1['dropout_rate'], child2['dropout_rate'] = \
            child2['dropout_rate'], child1['dropout_rate']
    
    if random.random() < 0.5:
        child1['learning_rate'], child2['learning_rate'] = \
            child2['learning_rate'], child1['learning_rate']
    
    if random.random() < 0.5:
        child1['optimizer'], child2['optimizer'] = \
            child2['optimizer'], child1['optimizer']
    
    # Crossover lists (random cut point)
    for list_key in ['filters', 'kernel_sizes', 'fc_nodes', 'activations']:
        if random.random() < 0.5:
            list1 = child1[list_key]
            list2 = child2[list_key]
            
            if len(list1) > 1 and len(list2) > 1:
                point1 = random.randint(1, len(list1) - 1)
                point2 = random.randint(1, len(list2) - 1)
                
                # Exchange parts
                new_list1 = list1[:point1] + list2[point2:]
                new_list2 = list2[:point2] + list1[point1:]
                
                child1[list_key] = new_list1
                child2[list_key] = new_list2
    
    # Assign new IDs
    child1['id'] = str(uuid.uuid4())[:8]
    child2['id'] = str(uuid.uuid4())[:8]
    child1['fitness'] = 0.0
    child2['fitness'] = 0.0
    
    return child1, child2

print("Genetic functions defined correctly")

## 6. Hybrid Neuroevolution Implementation

In [ ]:
class HybridNeuroevolution:
    """Main class that implements hybrid neuroevolution."""
    
    def __init__(self, config: dict, train_loader: DataLoader, test_loader: DataLoader):
        self.config = config
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.population = []
        self.generation = 0
        self.best_individual = None
        self.fitness_history = []
        self.generation_stats = []
        
    def initialize_population(self):
        """Initializes population with random genomes."""
        print(f"Initializing population of {self.config['population_size']} individuals...")
        
        self.population = []
        for i in range(self.config['population_size']):
            genome = create_random_genome(self.config)
            self.population.append(genome)
            
        print(f"Population initialized with {len(self.population)} individuals")
    
    def evaluate_fitness(self, genome: dict) -> float:
        """Evaluates genome fitness by training the neural network."""
        try:
            # Create model
            model = EvolvableCNN(genome, self.config).to(device)
            
            # Create optimizer
            optimizer_class = OPTIMIZERS[genome['optimizer']]
            optimizer = optimizer_class(model.parameters(), lr=genome['learning_rate'])
            
            # Loss function
            criterion = nn.CrossEntropyLoss()
            
            # Training
            model.train()
            total_train_batches = min(len(self.train_loader), self.config['early_stopping_patience'])
            
            print(f"      Training model {genome['id']} ({self.config['num_epochs']} epochs, max {total_train_batches} batches/epoch)")
            
            for epoch in range(self.config['num_epochs']):
                running_loss = 0.0
                batch_count = 0
                
                for data, target in self.train_loader:
                    data, target = data.to(device), target.to(device)
                    
                    optimizer.zero_grad()
                    output = model(data)
                    loss = criterion(output, target)
                    loss.backward()
                    optimizer.step()
                    
                    running_loss += loss.item()
                    batch_count += 1
                    
                    # Early stopping for quick evaluation
                    if batch_count >= self.config['early_stopping_patience']:
                        break
                
                # Epoch statistics
                avg_loss = running_loss / batch_count
                print(f"          Epoch {epoch+1}/{self.config['num_epochs']}: Average loss = {avg_loss:.4f} ({batch_count} batches processed)")
            
            print(f"      Training completed for model {genome['id']}")
            
            # Evaluation
            print(f"      Evaluating model {genome['id']} on test set...")
            model.eval()
            correct = 0
            total = 0
            eval_batches = 0
            max_eval_batches = min(len(self.test_loader), 20)
            total_eval_loss = 0.0
            
            with torch.no_grad():
                for data, target in self.test_loader:
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    
                    # Calculate evaluation loss
                    eval_loss = criterion(output, target)
                    total_eval_loss += eval_loss.item()
                    
                    # Calculate accuracy
                    _, predicted = torch.max(output, 1)
                    total += target.size(0)
                    correct += (predicted == target).sum().item()
                    eval_batches += 1
                    
                    # Early stopping in evaluation
                    if eval_batches >= max_eval_batches:
                        break
            
            accuracy = 100.0 * correct / total
            avg_eval_loss = total_eval_loss / eval_batches
            
            print(f"         Evaluation: {correct}/{total} correct = {accuracy:.2f}% accuracy")
            print(f"         Evaluation loss: {avg_eval_loss:.4f} ({eval_batches} batches evaluated)")
            print(f"      Evaluation completed for model {genome['id']} - Final fitness: {accuracy:.2f}%")
            
            return accuracy
            
        except Exception as e:
            print(f"      ERROR evaluating genome {genome['id']}: {e}")
            logger.warning(f"Error evaluating genome {genome['id']}: {e}")
            return 0.0
    
    def evaluate_population(self):
        """Evaluates entire population and updates fitness."""
        print(f"\nEvaluating population (Generation {self.generation})...")
        print(f"Processing {len(self.population)} individuals...")
        
        fitness_scores = []
        best_fitness_so_far = 0.0
        
        for i, genome in enumerate(self.population):
            print(f"\n   Evaluating individual {i+1}/{len(self.population)} (ID: {genome['id']})")
            print(f"      Architecture: {genome['num_conv_layers']} conv + {genome['num_fc_layers']} fc, opt={genome['optimizer']}, lr={genome['learning_rate']}")
            
            fitness = self.evaluate_fitness(genome)
            genome['fitness'] = fitness
            fitness_scores.append(fitness)
            
            # Update best fitness found so far
            if fitness > best_fitness_so_far:
                best_fitness_so_far = fitness
                print(f"      New best fitness in this generation: {fitness:.2f}%!")
            
            print(f"      Fitness obtained: {fitness:.2f}% | Best so far: {best_fitness_so_far:.2f}%")
        
        # Generation statistics
        avg_fitness = np.mean(fitness_scores)
        max_fitness = np.max(fitness_scores)
        min_fitness = np.min(fitness_scores)
        std_fitness = np.std(fitness_scores)
        
        stats = {
            'generation': self.generation,
            'avg_fitness': avg_fitness,
            'max_fitness': max_fitness,
            'min_fitness': min_fitness,
            'std_fitness': std_fitness
        }
        
        self.generation_stats.append(stats)
        self.fitness_history.append(max_fitness)
        
        # Update best individual
        best_genome = max(self.population, key=lambda x: x['fitness'])
        if self.best_individual is None or best_genome['fitness'] > self.best_individual['fitness']:
            self.best_individual = copy.deepcopy(best_genome)
            print(f"\nNew global best individual found!")
        
        print(f"\nGENERATION {self.generation} STATISTICS:")
        print(f"   Maximum fitness: {max_fitness:.2f}%")
        print(f"   Average fitness: {avg_fitness:.2f}%")
        print(f"   Minimum fitness: {min_fitness:.2f}%")
        print(f"   Standard deviation: {std_fitness:.2f}%")
        print(f"   Best individual: {best_genome['id']} with {best_genome['fitness']:.2f}%")
        print(f"   Global best individual: {self.best_individual['id']} with {self.best_individual['fitness']:.2f}%")
        
    def selection_and_reproduction(self):
        """Selects best individuals and creates new generation."""
        print(f"\nStarting selection and reproduction...")
        
        # Sort population by fitness
        self.population.sort(key=lambda x: x['fitness'], reverse=True)
        
        # Select elite
        elite_size = int(self.config['population_size'] * self.config['elite_percentage'])
        elite = self.population[:elite_size]
        
        print(f"Selecting {elite_size} elite individuals:")
        for i, individual in enumerate(elite):
            print(f"   Elite {i+1}: {individual['id']} (fitness: {individual['fitness']:.2f}%)")
        
        # Create new generation
        new_population = copy.deepcopy(elite)  # Preserve elite
        
        # Complete population with crossover and mutation
        offspring_needed = self.config['population_size'] - len(new_population)
        print(f"Creating {offspring_needed} new individuals through crossover and mutation...")
        
        offspring_created = 0
        while len(new_population) < self.config['population_size']:
            # Parent selection (tournament)
            parent1 = self.tournament_selection()
            parent2 = self.tournament_selection()
            
            # Crossover
            child1, child2 = crossover_genomes(parent1, parent2, self.config)
            
            # Mutation
            child1 = mutate_genome(child1, self.config)
            child2 = mutate_genome(child2, self.config)
            
            new_population.extend([child1, child2])
            offspring_created += 2
            
            if offspring_created % 4 == 0:  # Show progress every 4 individuals
                print(f"   Created {min(offspring_created, offspring_needed)} of {offspring_needed} new individuals...")
        
        # Adjust size if necessary
        self.population = new_population[:self.config['population_size']]
        
        print(f"New generation created with {len(self.population)} individuals")
        print(f"   Elite preserved: {elite_size}")
        print(f"   New individuals: {len(self.population) - elite_size}")
    
    def tournament_selection(self, tournament_size: int = 3) -> dict:
        """Tournament selection."""
        tournament = random.sample(self.population, min(tournament_size, len(self.population)))
        return max(tournament, key=lambda x: x['fitness'])
    
    def check_convergence(self) -> bool:
        """Checks if algorithm has converged."""
        # Check target fitness
        if self.best_individual and self.best_individual['fitness'] >= self.config['fitness_threshold']:
            print(f"Target fitness reached! ({self.best_individual['fitness']:.2f}% >= {self.config['fitness_threshold']}%)")
            return True
        
        # Check maximum generations
        if self.generation >= self.config['max_generations']:
            print(f"Maximum generations reached ({self.generation}/{self.config['max_generations']})")
            return True
        
        # Check stagnation (last 3 generations without significant improvement)
        if len(self.fitness_history) >= 3:
            recent_fitness = self.fitness_history[-3:]
            if max(recent_fitness) - min(recent_fitness) < 0.5:  # Less than 0.5% improvement
                print(f"Stagnation detected in last 3 generations")
                return True
        
        return False
    
    def evolve(self) -> dict:
        """Executes complete evolution process."""
        print("STARTING HYBRID NEUROEVOLUTION PROCESS")
        print("="*60)
        print(f"Configuration:")
        print(f"   Population: {self.config['population_size']} individuals")
        print(f"   Maximum generations: {self.config['max_generations']}")
        print(f"   Target fitness: {self.config['fitness_threshold']}%")
        print(f"   Device: {device}")
        print("="*60)
        
        # Initialize population
        self.initialize_population()
        
        # Main evolution loop
        while not self.check_convergence():
            print(f"\n{'='*80}")
            print(f"GENERATION {self.generation}")
            print(f"{'='*80}")
            
            # Evaluate population
            self.evaluate_population()
            
            # Check convergence before continuing
            if self.check_convergence():
                break
            
            # Selection and reproduction
            self.selection_and_reproduction()
            
            # Next generation
            self.generation += 1
            
            print(f"\nPreparing for next generation...")
        
        print(f"\n{'='*80}")
        print(f"EVOLUTION COMPLETED!")
        print(f"{'='*80}")
        print(f"Best individual found:")
        print(f"   ID: {self.best_individual['id']}")
        print(f"   Fitness: {self.best_individual['fitness']:.2f}%")
        print(f"   Origin generation: {self.generation}")
        print(f"   Total generations processed: {self.generation + 1}")
        
        return self.best_individual

print("HybridNeuroevolution class defined correctly (without tqdm, with detailed prints)")

## 7. Evolution Process Execution

In [ ]:
# Optional: Modify configuration to use custom dataset
# Uncomment and modify the following lines if you want to use a custom dataset
# CONFIG['use_custom_dataset'] = True
# CONFIG['dataset_path'] = r'E:\Neuroevolution\data\phd_data'  # Adjust path according to your dataset
# CONFIG['num_channels'] = 3  # 3 for RGB, 1 for grayscale
# CONFIG['num_classes'] = 2   # Adjust according to your number of classes

print("Current configuration:")
print(f"   Dataset: {'Custom' if CONFIG['use_custom_dataset'] else 'MNIST'}")
if CONFIG['use_custom_dataset']:
    print(f"   Path: {CONFIG['dataset_path']}")
print(f"   Population: {CONFIG['population_size']} individuals")
print(f"   Maximum generations: {CONFIG['max_generations']}")
print(f"   Target fitness: {CONFIG['fitness_threshold']}%")
print(f"   Device: {device}")

# Initialize neuroevolution system
start_time = datetime.now()
print(f"\nStarting neuroevolution at {start_time.strftime('%H:%M:%S')}")

# Create system instance
neuroevolution = HybridNeuroevolution(CONFIG, train_loader, test_loader)

# Execute evolution process
best_genome = neuroevolution.evolve()

end_time = datetime.now()
execution_time = end_time - start_time

print(f"\nProcess completed at {end_time.strftime('%H:%M:%S')}")
print(f"Total execution time: {execution_time}")
print(f"Total generations: {neuroevolution.generation}")
print(f"Best fitness achieved: {best_genome['fitness']:.2f}%")

## 8. Results Visualization and Analysis

In [ ]:
# Configure matplotlib style
plt.style.use('default')
sns.set_palette("husl")

# Function to visualize fitness evolution
def plot_fitness_evolution(neuroevolution):
    """Plots fitness evolution across generations."""
    if not neuroevolution.generation_stats:
        print("WARNING: No statistics data to plot")
        return
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Extract data and filter 0.00 fitness
    generations = []
    avg_fitness = []
    max_fitness = []
    min_fitness = []
    std_fitness = []
    
    for stat in neuroevolution.generation_stats:
        # Only include if valid fitness (> 0.00)
        if stat['max_fitness'] > 0.00:
            generations.append(stat['generation'])
            avg_fitness.append(stat['avg_fitness'])
            max_fitness.append(stat['max_fitness'])
            min_fitness.append(stat['min_fitness'])
            std_fitness.append(stat['std_fitness'])
    
    if not generations:
        print("WARNING: No valid fitness data to plot (all are 0.00)")
        return
    
    # Graph 1: Fitness evolution
    ax1.plot(generations, max_fitness, 'g-', linewidth=2, marker='o', label='Maximum Fitness')
    ax1.plot(generations, avg_fitness, 'b-', linewidth=2, marker='s', label='Average Fitness')
    ax1.plot(generations, min_fitness, 'r-', linewidth=2, marker='^', label='Minimum Fitness')
    ax1.fill_between(generations, 
                     [max(0, avg - std) for avg, std in zip(avg_fitness, std_fitness)],
                     [avg + std for avg, std in zip(avg_fitness, std_fitness)],
                     alpha=0.2, color='blue')
    
    ax1.set_xlabel('Generation')
    ax1.set_ylabel('Fitness (%)')
    ax1.set_title('Fitness Evolution by Generation (Excluding 0.00%)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add target fitness line
    ax1.axhline(y=CONFIG['fitness_threshold'], color='orange', linestyle='--', 
                label=f"Target ({CONFIG['fitness_threshold']}%)")
    ax1.legend()
    
    # Set Y axis limits for better visualization
    y_min = max(0, min(min_fitness) - 5)
    y_max = min(100, max(max_fitness) + 5)
    ax1.set_ylim(y_min, y_max)
    
    # Graph 2: Diversity (standard deviation)
    ax2.plot(generations, std_fitness, 'purple', linewidth=2, marker='D')
    ax2.set_xlabel('Generation')
    ax2.set_ylabel('Fitness Standard Deviation')
    ax2.set_title('Population Diversity (Excluding 0.00%)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Show additional information
    print(f"Plotted data:")
    print(f"   Generations with valid fitness: {len(generations)}")
    print(f"   Best fitness achieved: {max(max_fitness):.2f}%")
    print(f"   Final average fitness: {avg_fitness[-1]:.2f}%")
    if len(generations) < len(neuroevolution.generation_stats):
        excluded = len(neuroevolution.generation_stats) - len(generations)
        print(f"   WARNING: Excluded generations (0.00 fitness): {excluded}")

# Function to show detailed statistics
def show_evolution_statistics(neuroevolution):
    """Shows detailed evolution statistics."""
    print("DETAILED EVOLUTION STATISTICS")
    print("="*60)
    
    if not neuroevolution.generation_stats:
        print("WARNING: No statistics available")
        return
    
    # Filter statistics with valid fitness
    valid_stats = [stat for stat in neuroevolution.generation_stats if stat['max_fitness'] > 0.00]
    
    if not valid_stats:
        print("WARNING: No valid statistics (all fitness are 0.00)")
        return
    
    final_stats = valid_stats[-1]
    
    print(f"Completed generations: {neuroevolution.generation}")
    print(f"Generations with valid fitness: {len(valid_stats)}")
    if len(valid_stats) < len(neuroevolution.generation_stats):
        excluded = len(neuroevolution.generation_stats) - len(valid_stats)
        print(f"WARNING: Generations with 0.00 fitness (excluded): {excluded}")
    
    print(f"\nFINAL STATISTICS (excluding 0.00 fitness):")
    print(f"   Final best fitness: {final_stats['max_fitness']:.2f}%")
    print(f"   Final average fitness: {final_stats['avg_fitness']:.2f}%")
    print(f"   Final minimum fitness: {final_stats['min_fitness']:.2f}%")
    print(f"   Final standard deviation: {final_stats['std_fitness']:.2f}%")
    
    # Progress across generations
    if len(valid_stats) > 1:
        initial_max = valid_stats[0]['max_fitness']
        final_max = valid_stats[-1]['max_fitness']
        improvement = final_max - initial_max
        
        print(f"\nPROGRESS:")
        print(f"   Initial fitness: {initial_max:.2f}%")
        print(f"   Final fitness: {final_max:.2f}%")
        print(f"   Total improvement: {improvement:.2f}%")
        if initial_max > 0:
            print(f"   Relative improvement: {(improvement/initial_max)*100:.1f}%")
    
    # Convergence analysis
    print(f"\nCONVERGENCE CRITERIA:")
    if neuroevolution.best_individual and neuroevolution.best_individual['fitness'] >= CONFIG['fitness_threshold']:
        print(f"   OK: Target fitness reached ({CONFIG['fitness_threshold']}%)")
    else:
        print(f"   ERROR: Target fitness NOT reached ({CONFIG['fitness_threshold']}%)")
    
    if neuroevolution.generation >= CONFIG['max_generations']:
        print(f"   TIME: Maximum generations reached ({CONFIG['max_generations']})")
    
    # Additional performance statistics
    all_max_fitness = [stat['max_fitness'] for stat in valid_stats]
    all_avg_fitness = [stat['avg_fitness'] for stat in valid_stats]
    
    print(f"\nGENERAL STATISTICS:")
    print(f"   Best fitness of entire evolution: {max(all_max_fitness):.2f}%")
    print(f"   Average fitness of entire evolution: {np.mean(all_avg_fitness):.2f}%")
    print(f"   Average improvement per generation: {(max(all_max_fitness) - min(all_max_fitness))/len(valid_stats):.2f}%")
    
    if neuroevolution.best_individual:
        print(f"\nBest individual ID: {neuroevolution.best_individual['id']}")
        print(f"Best individual fitness: {neuroevolution.best_individual['fitness']:.2f}%")

# Additional function for failure analysis
def analyze_failed_evaluations(neuroevolution):
    """Analyzes evaluations that resulted in 0.00 fitness."""
    print("\nFAILED EVALUATIONS ANALYSIS")
    print("="*50)
    
    total_generations = len(neuroevolution.generation_stats)
    failed_generations = len([stat for stat in neuroevolution.generation_stats if stat['max_fitness'] == 0.00])
    
    if failed_generations == 0:
        print("OK: No failed evaluations (0.00 fitness)")
        return
    
    success_rate = ((total_generations - failed_generations) / total_generations) * 100
    
    print(f"Failure summary:")
    print(f"   Total generations: {total_generations}")
    print(f"   Failed generations: {failed_generations}")
    print(f"   Success rate: {success_rate:.1f}%")
    
    if failed_generations > 0:
        failed_gens = [stat['generation'] for stat in neuroevolution.generation_stats if stat['max_fitness'] == 0.00]
        print(f"   Generations with failures: {failed_gens}")
        
        print(f"\nPossible causes of 0.00 fitness:")
        print(f"   • Errors in model architecture")
        print(f"   • Memory problems (GPU/RAM)")
        print(f"   • Invalid hyperparameter configurations")
        print(f"   • Errors during training")

# Execute visualizations
plot_fitness_evolution(neuroevolution)
show_evolution_statistics(neuroevolution)
analyze_failed_evaluations(neuroevolution)

## 9. BEST ARCHITECTURE FOUND

In [ ]:
def display_best_architecture(best_genome, config):
    """
    Shows the best architecture found in detailed and visual format.
    """
    print("="*60)
    print("        BEST EVOLVED ARCHITECTURE")
    print("="*60)
    
    # General information
    print(f"\nGENERAL INFORMATION:")
    print(f"   Genome ID: {best_genome['id']}")
    print(f"   Fitness Achieved: {best_genome['fitness']:.2f}%")
    print(f"   Generation: {neuroevolution.generation}")
    
    # Architecture details
    print(f"\nNETWORK ARCHITECTURE:")
    print(f"   Convolutional Layers: {best_genome['num_conv_layers']}")
    print(f"   Fully Connected Layers: {best_genome['num_fc_layers']}")
    
    print(f"\nCONVOLUTIONAL LAYER DETAILS:")
    for i in range(best_genome['num_conv_layers']):
        filters = best_genome['filters'][i]
        kernel = best_genome['kernel_sizes'][i]
        activation = best_genome['activations'][i % len(best_genome['activations'])]
        print(f"   Conv{i+1}: {filters} filters, kernel {kernel}x{kernel}, activation {activation}")
    
    print(f"\nFULLY CONNECTED LAYER DETAILS:")
    for i, nodes in enumerate(best_genome['fc_nodes']):
        print(f"   FC{i+1}: {nodes} neurons")
    print(f"   Output: {config['num_classes']} neurons (classes)")
    
    print(f"\nHYPERPARAMETERS:")
    print(f"   Optimizer: {best_genome['optimizer'].upper()}")
    print(f"   Learning Rate: {best_genome['learning_rate']:.4f}")
    print(f"   Dropout Rate: {best_genome['dropout_rate']:.3f}")
    print(f"   Activation Functions: {', '.join(best_genome['activations'])}")
    
    # Create and show final model
    print(f"\nCREATING FINAL MODEL...")
    try:
        final_model = EvolvableCNN(best_genome, config)
        total_params = sum(p.numel() for p in final_model.parameters())
        trainable_params = sum(p.numel() for p in final_model.parameters() if p.requires_grad)
        
        print(f"   Model created successfully")
        print(f"   Total parameters: {total_params:,}")
        print(f"   Trainable parameters: {trainable_params:,}")
        
        # Architecture summary
        print(f"\nCOMPACT SUMMARY:")
        print(f"   {final_model.get_architecture_summary()}")
        
    except Exception as e:
        print(f"   ERROR creating model: {e}")
    
    # Visualization in table format
    print(f"\nSUMMARY TABLE:")
    print(f"{'='*80}")
    print(f"{'Parameter':<25} {'Value':<30} {'Description':<25}")
    print(f"{'='*80}")
    print(f"{'ID':<25} {best_genome['id']:<30} {'Unique identifier':<25}")
    print(f"{'Fitness':<25} {best_genome['fitness']:.2f}%{'':<25} {'Accuracy achieved':<25}")
    print(f"{'Conv Layers':<25} {best_genome['num_conv_layers']:<30} {'Convolutional layers':<25}")
    print(f"{'FC Layers':<25} {best_genome['num_fc_layers']:<30} {'FC layers':<25}")
    print(f"{'Optimizer':<25} {best_genome['optimizer']:<30} {'Optimization algorithm':<25}")
    print(f"{'Learning Rate':<25} {best_genome['learning_rate']:<30} {'Learning rate':<25}")
    print(f"{'Dropout':<25} {best_genome['dropout_rate']:<30} {'Dropout rate':<25}")
    print(f"{'='*80}")
    
    # Comparison with initial configuration
    print(f"\nCOMPARISON WITH OBJECTIVES:")
    if best_genome['fitness'] >= config['fitness_threshold']:
        print(f"   TARGET: OK Fitness objective REACHED ({best_genome['fitness']:.2f}% >= {config['fitness_threshold']}%)")
    else:
        print(f"   TARGET: ERROR Fitness objective NOT reached ({best_genome['fitness']:.2f}% < {config['fitness_threshold']}%)")
    
    print(f"   TIME: Generations used: {neuroevolution.generation}/{config['max_generations']}")
    
    # Save information to JSON
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"best_architecture_{timestamp}.json"
    
    results_data = {
        'timestamp': timestamp,
        'execution_time': str(execution_time),
        'config_used': config,
        'best_genome': best_genome,
        'final_generation': neuroevolution.generation,
        'evolution_stats': neuroevolution.generation_stats
    }
    
    try:
        with open(results_file, 'w') as f:
            json.dump(results_data, f, indent=2, default=str)
        print(f"\nResults saved to: {results_file}")
    except Exception as e:
        print(f"\nWARNING: Error saving results: {e}")
    
    print(f"\nHYBRID NEUROEVOLUTION COMPLETED SUCCESSFULLY!")
    print(f"{'='*60}")

# Show the best architecture found
display_best_architecture(best_genome, CONFIG)